In [193]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import pandas as pd
import os
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import requests
from datetime import date
import time
from csv import writer

try:
    # Close any existing WebDrivers
    driver.close() 
except Exception:
    pass

#setting up driver
s=Service('C:\\Users\Korisnik\PycharmProjects\soupokusaji\chromedriver.exe')
driver = webdriver.Chrome(service=s)

#defining waittime for driver
waittime = 20

In [199]:
# function that goes through displayed results and scrapes data about facilities and available units
def facilityscrape ():
    # finding facility (entity) codes(numbers) that will be later used to create a unique xpath for "view rates" button
    entities = driver.find_elements(By.NAME, 'EntityNumber')
    
    # saving these numbers in a list to loop through facilities
    facilities = []
    for entity in entities[1::2]:
        facilities.append(entity.get_attribute('value'))

    # check if facility is already scraped, if not then add its entity number to list of unique facilities
    for facility in facilities:
        if facility not in unique_facilities:
            unique_facilities.append(facility)
            WebDriverWait(driver, waittime).until(EC.presence_of_element_located((By.XPATH, '//*[@id="viewRates_' + facility + '"]'))).click()
            
            # scraping facility's url
            get_url = driver.current_url
            
            # accessing html
            html_text = requests.get(get_url).text
            soup = BeautifulSoup(html_text, 'lxml')

            # scraping facility's address
            
            # addresses may contain additional instructions inside parentheses, they are unnecessary so they're disposed of
            address_ = soup.find('address').text
            if "(" in address_:
                address_, x1 = address_.split("(")
            
            # adresses contain weird blank spaces that are easiest to get rid of by splitting the string and joining it again
            splitt= address_.split()
            address_= ' '.join(splitt)
            
            # splitting address into street, state and zipcode
            occur = address_.count(',')
            if occur == 1:
                address_, state = address_.split(",")
            elif occur == 2:
                address_, region, state = address_.split(",")
                address_ = address_ + region
            elif occur == 0:
                state = "- -"
            else:
                add1, add2, region, state = address_.split(",")
                address_ = add1 + add2 + region
                

            state, zipp = state.split()           
            

            # scraping facility's name
            
            # if facility is u-haul's affiliate, its name is formatted differently from other facilities,
            # so the code first checks if there exists name formatted like that
            if driver.find_elements(By.XPATH, '//*[@id="mainRow"]/div/div/div/div/div/div[1]/h1'):
                
                name_ = soup.find('h1', class_="collapse").text
                name_ = name_.strip()
                print(name_)

            else:
                name_ = soup.find('h2', class_="collapse-half text-dull text-xl text-semibold").text
                name_ = name_.strip()
            

            # getting todays date
            today = date.today()
            thedate = today.strftime("%Y-%m-%d")

            #facility features will contain date, facility id, name, address and url
            fac_features = []
            
            value0 = thedate
            value1 = facility
            value2 = name_
            value3 = address_
            value4 = state
            value5 = zipp
            value6 = get_url
            
            fac_features.append(value0)
            fac_features.append(value1)
            fac_features.append(value2)
            fac_features.append(value3)
            fac_features.append(value4)
            fac_features.append(value5)
            fac_features.append(value6)


            facility_masterlist.append(fac_features)


            # finding all displayed units
            units = soup.find_all('div', class_="grid-x grid-margin-x align-left medium-grid-expand-x large-align-middle")
            
            # looping through units
            for unit in units:
                
                # scraping unit description
                unit_desc = unit.p.text
                
                # excluding units that aren't the regular self storage units - RV/Boat, lockers, wine, office and warehouse
                if "RV/Boat" not in unit_desc and "Lockers" not in unit_desc and "Office" not in unit_desc and "Wine" not in unit_desc and "Warehouse" not in unit_desc:
                    
                    # scraping unit size and splitting it into description (small, medium, large) and
                    # actual size in numbers 
                    unit_size = unit.h4.text.strip()
                    size_desc, size = unit_size.split(" | ")
                    
                    # scraping unit price
                    unit_price = unit.find('b', class_="text-lg").text.strip()
                    unit_price =unit_price.strip("$")
                    
                    # formatting unit description to get rid of blank spaces
                    splitted= unit_desc.split()
                    unit_desc= ' '.join(splitted)


                    # saving scraped data one by one to storage_features list, then appending that list
                    # as one row to units_masterlist
                    storage_features = []

                    value_1 = thedate
                    value_2 = facility
                    value_3 = unit_idd[0]
                    value_4 = size_desc
                    value_5 = size
                    value_6 = unit_desc
                    value_7 = unit_price

                    storage_features.append(value_1)
                    storage_features.append(value_2)
                    storage_features.append(value_3)
                    storage_features.append(value_4)
                    storage_features.append(value_5)
                    storage_features.append(value_6)
                    storage_features.append(value_7)
                    
                    units_masterlist.append(storage_features)


                    # increasing number for unit id
                    unit_idd[0]+= 1
                        

            #goes back to results page to scrape the next facility from results
            driver.back()

    
    


In [195]:
# function that goes through 4 pages of results
def resultsloop ():   
            # invoking function that scrapes the results
            facilityscrape()
            # making a pause in order not to overwhelm the website's server
            time.sleep(1.5)
            
            # moving to the next 6 results
            # for some cities there is less than 4 pages, so the function checks if there is "more locations" button
            if driver.find_elements(By.XPATH, '//*[@id="locationsResults"]/ul[2]/li/a[2]'):
                
                WebDriverWait(driver, waittime).until(EC.presence_of_element_located((By.XPATH, '//*[@id="locationsResults"]/ul[2]/li/a[2]'))).click()
                # invoking function that scrapes the results 
                facilityscrape()
                time.sleep(1.5)
                
                # checking if there is "more locations" button before moving onto next 6 results
                if driver.find_elements(By.XPATH, '//*[@id="locationsResults"]/ul[2]/li[2]/a[2]'):

                    WebDriverWait(driver, waittime).until(EC.presence_of_element_located((By.XPATH, '//*[@id="locationsResults"]/ul[2]/li[2]/a[2]'))).click()
                    # invoking function that scrapes the results 
                    facilityscrape()
                    time.sleep(1.5)

                    # checking if there is "more locations" button before moving onto last 2 results
                    if driver.find_elements(By.XPATH, '//*[@id="locationsResults"]/ul[2]/li[2]/a[2]'):
                    
                        WebDriverWait(driver, waittime).until(EC.presence_of_element_located((By.XPATH, '//*[@id="locationsResults"]/ul[2]/li[2]/a[2]'))).click()
                        facilityscrape()
                        time.sleep(1.5)

            
            
# function that goes through the list of cities, searches for the selected city and calls function for scraping            
def cityloop(city):

    try:
        
        # checks if there is a search form on the page, in some circumstances due to previous city malfunction there 
        # won't be a search form, so this covers that case to prevent code from looping through cities without
        # scraping any data
        if driver.find_elements(By.XPATH, '//*[@id="locationSearchForm"]/div[2]/div[2]/button'):
            WebDriverWait(driver, waittime).until(EC.presence_of_element_located((By.XPATH, '//*[@id="movingFromInput"]'))).clear()
            # insert text in search bar
            WebDriverWait(driver, waittime).until(EC.presence_of_element_located((By.XPATH, '//*[@id="movingFromInput"]'))).send_keys(city)
            # click submit to search for city 
            WebDriverWait(driver, waittime).until(EC.presence_of_element_located((By.XPATH, '//*[@id="locationSearchForm"]/div[2]/div[2]/button'))).click()
            
            # in case there isn't an exact match to city+state code, it will move on to the next city
            if driver.find_elements(By.XPATH, '//*[@id="locationAmbigAddresses"]/p') or driver.find_elements(By.XPATH, '//*[@id="locationSearchForm"]/fieldset/div/div[1]/ul/li'):
                print("would've thrown an error but im nice " + city)
                url='https://www.uhaul.com/Storage/WY/Results/'
                driver.get(url)
            else:
                # if the search is successful, this invokes function that scrapes the results
                resultsloop()
                print("finished scraping " + city)
        
        #in case of malfunction, sets up the page with valid search form for next city 
        else:
            print("something is wrong with (probably couldn't scrape city before): " + city)
            url='https://www.uhaul.com/Storage/WY/Results/'
            driver.get(url)
            
            

         
            
    # in case of exception
    except Exception:
        print("didnt fully scrape " + city)


In [196]:
# creating a list that will hold all facility data, and will be used to create a csv file
facility_masterlist = []
# creating a list that will have all scraped units, and will be used to create a csv file
units_masterlist=[]
# creating a set of entity numbers so that duplicates aren't scraped
unique_facilities = []
# setting up internal unit id
unit_idd=[0]
# opening uhaul storage domain, specifically results for wyoming since it's easier
# to search for cities from the results page of some other city/state
url='https://www.uhaul.com/Storage/WY/Results/'
driver.get(url)


In [ ]:
# opening an excel file containing a list of all major cities in the US, list is taken from Britannica
df = pd.read_excel('all_states.xlsx')    

# taking city + state code and feeding it to the function that will loop through the cities and scrape results
result = df['city'] + " " + df['state_code']
result.apply(cityloop)

#saving facility master list as csv file with unique name containing today's date
df2 = pd.DataFrame(facility_masterlist, columns=['date', 'facility_id', 'name', 'address', 'region', 'postal_code', 'url'])
today = date.today()
thedate = today.strftime("%d-%m-%Y")
namee = 'facilities-' + thedate + ".csv"
df2.to_csv(namee, index=False, sep=',')
print('Created facility skrejp')

#saving units master list as csv file with unique name containing today's date
df3 = pd.DataFrame(units_masterlist, columns=['date', 'location_external_id', 'internal_id', 'size_description', 'size', 'description', 'price'])
namee_ = 'units-' + thedate + ".csv"
df3.to_csv(namee_, index=False, sep=',')
print('Created new master list file')